In [11]:
import requests
import json
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import math

def get_charging_stations(latitude, longitude, radius):
    """
    Fetches charging stations within a given radius using the Chargefox API.

    Args:
        latitude: Latitude of the center point.
        longitude: Longitude of the center point.
        radius: Radius in kilometers.

    Returns:
        List of charging station coordinates, or an empty list if none are found.
    """
    url = "https://api.chargefox.com/v1/chargers/search"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "access_token": "YOUR_CHARGEFOX_API_KEY"  # Replace with your API key
    }

    response = requests.get(url, params=params)
    data = json.loads(response.text)

    if 'chargers' in data:
        charging_stations = [(charger['location']['coordinates'][1], charger['location']['coordinates'][0]) for charger in data['chargers']]
        return charging_stations
    else:
        return []

def calculate_route(origin, destination, max_range, starting_range, buffer_range, charging_stations):
    """
    Calculates the route with EV charging stops.

    Args:
        origin: Tuple of (latitude, longitude) for the origin.
        destination: Tuple of (latitude, longitude) for the destination.
        max_range: Maximum distance the EV can travel on a full charge.
        starting_range: Distance the EV can travel with the current charge.
        buffer_range: Safety buffer to ensure the vehicle doesn't run out before reaching the next station.
        charging_stations: List of charging station coordinates.

    Returns:
        List of tuples of (latitude, longitude) representing the route with charging stops.
    """
    if not charging_stations:
        print("No charging stations available.")
        return None

    route = [origin]
    current_location = origin
    current_range = starting_range

    while geodesic(current_location, destination).km > current_range - buffer_range:
        nearest_station = min(charging_stations, key=lambda station: geodesic(current_location, station).km)
        distance_to_station = geodesic(current_location, nearest_station).km

        if distance_to_station > current_range - buffer_range:
            print(f"Cannot reach station at {nearest_station} from {current_location}.")
            return None

        route.append(nearest_station)
        print(f"Added charging stop at: {nearest_station}, remaining range reset to: {max_range} km")
        current_location = nearest_station
        current_range = max_range

    route.append(destination)
    return route

# Example usage
geolocator = Nominatim(user_agent="my_app")
origin_address = "Melbourne, Australia"
destination_address = "Sydney, Australia"

origin = geolocator.geocode(origin_address)
destination = geolocator.geocode(destination_address)

if origin and destination:
    origin_coords = (origin.latitude, origin.longitude)
    destination_coords = (destination.latitude, destination.longitude)

    max_range = 300  # km
    starting_range = 200  # km
    buffer_range = 20  # km

    # Get charging stations for origin and destination
    origin_stations = get_charging_stations(origin.latitude, origin.longitude, 100)
    destination_stations = get_charging_stations(destination.latitude, destination.longitude, 100)
    all_stations = origin_stations + destination_stations

    if not all_stations:
        print("No charging stations found in the specified radius.")
    else:
        route = calculate_route(origin_coords, destination_coords, max_range, starting_range, buffer_range, all_stations)
        if route:
            print("Route with charging stops:", route)
        else:
            print("Unable to find a feasible route with the given parameters.")
else:
    print("Error: Unable to geocode origin or destination address.")


No charging stations found in the specified radius.
